# Introduction

This notebook provides a baseline for each setting in [Subtask A of SemEval 2022 Task 2](https://sites.google.com/view/semeval2022task2-idiomaticity#h.qq7eefmehqf9). In addition this provides some helpful pre-processing scripts that you are free to use with your experiments. 

Please start by stepping through this notebook so you have a clear idea as to what is expected of the task and what you need to submit. 

These baselines are based on the results described in the paper “[AStitchInLanguageModels: Dataset and Methods for the Exploration of Idiomaticity in Pre-Trained Language Models](https://arxiv.org/abs/2109.04413)”. 

## Zero-shot setting: Methodology 

Note that in the zero-shot setting you are NOT allowed to train the model using the one-shot data. 

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). This is based on the results presented in the dataset paper. 

We use Multilingual BERT for this setting.

## One-shot setting: Methodology

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. Again, this is based on the results presented in the dataset paper. 

We also use Multilingual BERT for this setting.


# Setup 

In [1]:
%load_ext autoreload
%autoreload 2

Download the Task data and evaluation scripts

In [2]:
#!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Download the “AStitchInLanguageModels” code which we make use of. 

In [3]:
#!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Download and install an editable version of huggingfaces transformers. 

In [4]:
#!git clone https://github.com/huggingface/transformers.git
#%cd transformers/
#!pip install --editable .
#%cd /content/ 

Required for run_glue ... 

In [5]:
## run_glue needs this. 
!pip install datasets

You should consider upgrading via the '/scratch/project_2001839/idioms/gutenfin/venv/bin/python3 -m pip install --upgrade pip' command.


Editable install requires runtime restart unless we do this. 

In [6]:
import site
site.main()


# Imports and Helper functions

In [7]:
import numpy as np

In [8]:
import os
import csv

from pathlib import Path

In [9]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data


In [10]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


The following function creates a submission file from the predictions output by run_glue (the text classification script from huggingface transformers - see below). 

Note that we set it up so we can load up results for only one setting. 

It requires as input the submission format file, which is available with the data. You can call this after completing each setting to load up results for both settings (see below).


In [11]:
def insert_to_submission_file( submission_format_file, input_file, prediction_format_file, setting ) :
    submission_header, submission_content = load_csv( submission_format_file )
    input_header     , input_data         = load_csv( input_file             )
    prediction_header, prediction_data    = load_csv( prediction_format_file, '\t' )

    assert len( input_data ) == len( prediction_data )

    ## submission_header ['ID', 'Language', 'Setting', 'Label']
    ## input_header      ['label', 'sentence1' ]
    ## prediction_header ['index', 'prediction']

    prediction_data = list( reversed( prediction_data ) )

    started_insert  = False
    for elem in submission_content : 
        if elem[ submission_header.index( 'Setting' ) ] != setting :
            if started_insert :
                if len( prediction_data ) == 0 :
                    break
                else : 
                    raise Exception( "Update should to contiguous ... something wrong." ) 
            continue
        started_insert = True
        elem[ submission_header.index( 'Label' ) ] = prediction_data.pop()[ prediction_header.index( 'prediction' ) ]

    return [ submission_header ] + submission_content

# Pre-process: Create train and dev and evaluation data in required format

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). 

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. 


## Functions for pre-processing

### _get_train_data

This function generates training data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 


In [12]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

### _get_dev_eval_data

This function generates training dev and eval data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 

Additionally, if there is no gold label provides (as in the case of eval) it will generate a file that can be used to generate predictions.


In [13]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data


### create_data

This function generates the training, development and evaluation data. 


In [14]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location ) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False
    )
    write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = True,
        include_idiom    = False
    )        
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )

    test_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )
    

    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

In [15]:
def create_test_data( input_location, output_location ) :

    test_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'test.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( test_data, os.path.join( output_location, 'ZeroShot', 'test.csv' ) )

    return

## Setup and Create data

In [16]:
!ls 

AStitchInLanguageModels  LICENSE	     requirements.txt
data			 make_submission.sh  SemEval_2022_Task2-idiomaticity
Data			 models		     SubTaskA-bert.ipynb
jupyter.10140283.out	 outputs	     SubTaskA-zeroshot.ipynb
jupyter.10230563.out	 pyproject.toml      SubTaskA-zeroshot-loaded.ipynb
lib			 README.md	     xlm-sentiment


In [17]:
outpath = 'Data'

In [ ]:
Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'ZeroShotPlus' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath )

In [ ]:
create_test_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData/', outpath )

# Zero Shot Setting

## Train Zero shot

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
## Create save path
#!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/
## Copy saved model.
#!cp -r /content/models/ZeroShot/0/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/

In [ ]:
## Bring back saved model here. 
#!mkdir -p /content/models/ZeroShot/0/
# !cp -r /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/* /content/models/ZeroShot/0/

## Evaluation On Dev Data

In [18]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/0' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

01/17/2022 13:28:44 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/17/2022 13:28:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_m

### Use predictions to create the submission file (for dev data)

In [ ]:
params = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/0/eval-dev/test_results_None.txt'                        ,
    }
params[ 'setting' ] = 'zero_shot'

In [ ]:
 updated_data = insert_to_submission_file( **params )

In [ ]:
!mkdir -p outputs

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_dev_formated.csv' ) 

### For the development data, we can run evaluation script.

In [19]:
import sys
sys.path.append( 'SemEval_2022_Task2-idiomaticity/SubTaskA/' ) 
from SubTask1Evaluator import evaluate_submission


submission_file = 'outputs/zero_shot_dev_formated.csv'
gold_file       = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'

results = evaluate_submission( submission_file, gold_file )
#%reload_ext google.colab.data_table
import pandas as pd
df = pd.DataFrame(data=results[1:], columns=results[0])
df

,Settings,Languages,F1 Score (Macro)
0,zero_shot,EN,0.70485
1,zero_shot,PT,0.611721
2,zero_shot,"EN,PT",0.6939
3,one_shot,EN,"(None, None, None)"
4,one_shot,PT,"(None, None, None)"
5,one_shot,"EN,PT","(None, None, None)"


### Creating separate models for English and multi-lingual

In [20]:
from lib import util

In [21]:
frames = util.load_csv_dataframes('SemEval_2022_Task2-idiomaticity/SubTaskA/Data')
tframes = util.load_csv_dataframes('SemEval_2022_Task2-idiomaticity/SubTaskA/TestData')

In [22]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']
tdf = tframes['test.csv']

In [23]:
zdf_enindex = zdf['Language'] == 'EN'
dev_enindex = ddf['Language'] == 'EN'
eval_enindex = edf['Language'] == 'EN'
test_enindex = tdf['Language'] == 'EN'

In [24]:
zdf_en = zdf[zdf_enindex].drop(['Language', 'DataID', 'Setting', 'Previous', 'Next'],
                                axis=1).rename(columns={'Label': 'label',
                                                        'Target': 'sentence1',
                                                        'MWE': 'sentence2'})

In [25]:
zdf_oth = zdf[~zdf_enindex].drop(['Language', 'DataID', 'Setting', 'Previous', 'Next'],
                                 axis=1).rename(columns={'Label': 'label',
                                                         'Target': 'sentence1',
                                                         'MWE': 'sentence2'})

In [26]:
ddf['label'] = ddf_gold['Label']

In [27]:
ddf_en = ddf[dev_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})
ddf_oth = ddf[~dev_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                 axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})

In [28]:
edf['label'] = 1

In [29]:
edf_en = edf[eval_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                 axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})
edf_oth = edf[~eval_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                   axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})

In [30]:
tdf['label'] = 1

In [31]:
tdf_en = tdf[test_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                 axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})
tdf_oth = tdf[~test_enindex].drop(['Language', 'ID', 'Previous', 'Next'],
                                   axis=1).rename(columns={'Target': 'sentence1', 'MWE': 'sentence2'})

In [ ]:
zdf_en.to_csv('Data/ZeroShot/train_en_2.csv', index=False)
zdf_oth.to_csv('Data/ZeroShot/train_ot_2.csv', index=False)

In [ ]:
ddf_en.to_csv('Data/ZeroShot/dev_en_2.csv', index=False)
ddf_oth.to_csv('Data/ZeroShot/dev_ot_2.csv', index=False)

In [ ]:
edf_en.to_csv('Data/ZeroShot/eval_en_2.csv', index=False)
edf_oth.to_csv('Data/ZeroShot/eval_ot_2.csv', index=False)

In [ ]:
tdf_en.to_csv('Data/ZeroShot/test_en.csv', index=False)
tdf_oth.to_csv('Data/ZeroShot/test_ot.csv', index=False)

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/1/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en_2.csv \
    	--validation_file Data/ZeroShot/dev_en_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_ot_2.csv \
    	--validation_file Data/ZeroShot/dev_ot_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

### Evaluation for language-separated data

In [32]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/1' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/1/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en_2.csv \
    	--validation_file Data/ZeroShot/dev_en_2.csv \
      --test_file Data/ZeroShot/dev_en_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

01/17/2022 13:30:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/17/2022 13:30:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_m

01/17/2022 13:31:00 - INFO - __main__ -   ***** Test results None *****
100%|███████████████████████████████████████████| 59/59 [00:01<00:00, 39.01it/s]


In [33]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/2' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_ot_2.csv \
    	--validation_file Data/ZeroShot/dev_ot_2.csv \
      --test_file Data/ZeroShot/dev_ot_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

01/17/2022 13:31:03 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/17/2022 13:31:03 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_m

01/17/2022 13:31:14 - INFO - __main__ -   ***** Test results None *****
100%|███████████████████████████████████████████| 35/35 [00:00<00:00, 42.98it/s]


Combine EN+PT results to Comb.

In [ ]:
!awk 'FNR==1 && NR!=1{next;}{print}' models/ZeroShot/1/eval-dev/test_results_None.txt models/ZeroShot/2/eval-dev/test_results_None.txt > models/ZeroShot/1/eval-dev/test_results_Comb.txt

In [ ]:
params_z = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/1/eval-dev/test_results_Comb.txt'                        ,
    }
params_z[ 'setting' ] = 'zero_shot'

In [ ]:
updated_data = insert_to_submission_file( **params_z )

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_dev_formated_comb.csv' ) 

In [34]:
results_comb = evaluate_submission( 'outputs/zero_shot_dev_formated_comb.csv', gold_file )
#%reload_ext google.colab.data_table
pd.DataFrame(data=results_comb[1:], columns=results_comb[0])

,Settings,Languages,F1 Score (Macro)
0,zero_shot,EN,0.760457
1,zero_shot,PT,0.590239
2,zero_shot,"EN,PT",0.725061
3,one_shot,EN,"(None, None, None)"
4,one_shot,PT,"(None, None, None)"
5,one_shot,"EN,PT","(None, None, None)"


Get the English results from the English model and Portuguese results from the full model.

In [36]:
dres_0 = util.load_df('models/ZeroShot/0/eval-dev/test_results_None.txt', delimiter="\t")
dres_1 = util.load_df('models/ZeroShot/1/eval-dev/test_results_None.txt', delimiter="\t")

In [37]:
dres_0['index'] = dres_0['index'].astype(int)
dres_1['index'] = dres_1['index'].astype(int)

In [38]:
for i, row in dres_1.iterrows():
    # print(i, row)
    dres_0.loc[i, 'prediction'] = row['prediction']
    # break

In [ ]:
dres_0.to_csv('models/ZeroShot/1/eval-dev/test_results_Comb2.txt', index=False, sep="\t")

In [ ]:
params_z2 = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/1/eval-dev/test_results_Comb2.txt'                        ,
    }
params_z2[ 'setting' ] = 'zero_shot'

In [ ]:
updated_data = insert_to_submission_file( **params_z2 )
write_csv( updated_data, 'outputs/zero_shot_dev_formated_comb2.csv' ) 

In [39]:
results_comb2 = evaluate_submission( 'outputs/zero_shot_dev_formated_comb2.csv', gold_file )
#%reload_ext google.colab.data_table
pd.DataFrame(data=results_comb2[1:], columns=results_comb2[0])

,Settings,Languages,F1 Score (Macro)
0,zero_shot,EN,0.760457
1,zero_shot,PT,0.611721
2,zero_shot,"EN,PT",0.721204
3,one_shot,EN,"(None, None, None)"
4,one_shot,PT,"(None, None, None)"
5,one_shot,"EN,PT","(None, None, None)"


In the end, getting the PT results from the full model didn't seem to improve things.

## Generate Eval Data output

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/0' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/eval-eval/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/eval.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

### Use predictions to create the submission file (for eval data)

In [ ]:
params = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/0/eval-eval/test_results_None.txt'                        ,
    }
params[ 'setting' ] = 'zero_shot'

In [ ]:
 updated_data = insert_to_submission_file( **params )

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_eval_formated.csv' ) 

### Use language-specific models.

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/1' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/1/eval-eval/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en_2.csv \
    	--validation_file Data/ZeroShot/dev_en_2.csv \
      --test_file Data/ZeroShot/eval_en_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/2' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/eval-eval/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_ot_2.csv \
    	--validation_file Data/ZeroShot/dev_ot_2.csv \
      --test_file Data/ZeroShot/eval_ot_2.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!awk 'FNR==1 && NR!=1{next;}{print}' models/ZeroShot/1/eval-eval/test_results_None.txt models/ZeroShot/2/eval-eval/test_results_None.txt > models/ZeroShot/1/eval-eval/test_results_Comb2.txt


In [ ]:
params_ez = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/1/eval-eval/test_results_Comb2.txt'                        ,
    }
params_ez[ 'setting' ] = 'zero_shot'

In [ ]:
 updated_data = insert_to_submission_file( **params_ez )

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_eval_formated_comb.csv' ) 

**NOTE**: You can submit this file, but it only has results for the zero-shot setting.

# Test data

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/0' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/eval-test/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/test.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
tparams = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData/test_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData/test.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/0/eval-test/test_results_None.txt'                        ,
    }
tparams[ 'setting' ] = 'zero_shot'

In [ ]:
updated_data = insert_to_submission_file( **tparams )

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_test_formated.csv' ) 

### Use language-specific models.

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/1' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/1/eval-test/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en_2.csv \
    	--validation_file Data/ZeroShot/dev_en_2.csv \
      --test_file Data/ZeroShot/test_en.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/ZeroShot/2' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/eval-test/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_ot_2.csv \
    	--validation_file Data/ZeroShot/dev_ot_2.csv \
      --test_file Data/ZeroShot/test_ot.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!awk 'FNR==1 && NR!=1{next;}{print}' models/ZeroShot/1/eval-test/test_results_None.txt models/ZeroShot/2/eval-test/test_results_None.txt > models/ZeroShot/1/eval-test/test_results_Comb2.txt


In [ ]:
t2params = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData/test_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData/test.csv'                   ,
    'prediction_format_file' : 'models/ZeroShot/1/eval-test/test_results_Comb2.txt'                        ,
    }
t2params[ 'setting' ] = 'zero_shot'

In [ ]:
updated_data = insert_to_submission_file( **t2params )

In [ ]:
write_csv( updated_data, 'outputs/zero_shot_test_formated_comb.csv' ) 

# One Shot Setting

## Train One shot

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
## Create save path
#!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/OneShot/1/
## Copy saved model.
#!cp -r /content/models/OneShot/1/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/OneShot/1/

## Evaluation On Dev Data

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/OneShot/1' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/eval-dev/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
      --test_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

### Use predictions to create the submission file (for dev data)

In [ ]:
params = {
    'submission_format_file' : 'outputs/zero_shot_dev_formated.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : 'models/OneShot/1/eval-dev/test_results_None.txt'                        ,
    }
params[ 'setting' ] = 'one_shot'

In [ ]:
 updated_data = insert_to_submission_file( **params )
 write_csv( updated_data, 'outputs/both_dev_formated.csv' ) 

### For the development data, we can run evaluation script.

In [ ]:
import sys
sys.path.append( 'SemEval_2022_Task2-idiomaticity/SubTaskA/' ) 
from SubTask1Evaluator import evaluate_submission


submission_file = 'outputs/both_dev_formated.csv'
gold_file       = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'

results = evaluate_submission( submission_file, gold_file )
#%reload_ext google.colab.data_table
import pandas as pd
df = pd.DataFrame(data=results[1:], columns=results[0])
df

## Generate Eval Data output

In [ ]:
!python AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'models/OneShot/1' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/eval-eval/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
      --test_file Data/OneShot/eval.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

### Use predictions to create the submission file (for eval data)

#### Create One Shot submission

In [ ]:
params = {
    'submission_format_file' : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval_submission_format.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval.csv'                   ,
    'prediction_format_file' : 'models/OneShot/1/eval-eval/test_results_None.txt'                         ,
    }
params[ 'setting' ] = 'one_shot'


In [ ]:
 updated_data = insert_to_submission_file( **params )
 write_csv( updated_data, 'outputs/one_shot_eval_formated.csv' ) 

#### Combine Zero Shot and One Shot submission files.

Do this by loading zero shot data as submission file format.

In [ ]:
params = {
    'submission_format_file' : 'outputs/zero_shot_eval_formated.csv' ,
    'input_file'             : 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval.csv'                   ,
    'prediction_format_file' : 'models/OneShot/1/eval-eval/test_results_None.txt'                        ,
    }
params[ 'setting' ] = 'one_shot'


In [ ]:
 updated_data = insert_to_submission_file( **params )
 write_csv( updated_data, 'outputs/task2_subtaska.csv' ) 

# Download Submission File

In [ ]:
from google.colab import files
files.download('/content/outputs/task2_subtaska.csv') 
## Remeber to put this in a folder called "submission".

# Discussion

In [ ]:
df

Notice the significant jump in F1 scores with the introduction of just one positive and one negative example. 

Note that your position on the leaderboard will be based on rows with index 2 and 5 (combined results for both languages). The rest of the results for information and ablation studies. 



